<a href="https://colab.research.google.com/github/truongtaoday/train/blob/main/SDVN_TrainingSDXL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [![](https://img.shields.io/badge/Thiết%20kế-stablediffusion.vn-0075ff)](https://stablediffusion.vn) [![](https://img.shields.io/badge/Phiên%20bản-v3-0075ff)](https://stablediffusion.vn) [![](https://img.shields.io/badge/Legacy-Version-ff0000)](https://bit.ly/trainsdxl) [![](https://img.shields.io/badge/Group-Support-0075ff)](https://www.facebook.com/groups/stablediffusion.vn) [![](https://img.shields.io/badge/Bộ%20công%20cụ-Đầy%20đủ-0075ff)](https://stablediffusion.vn/bo-cong-cu/) [![](https://img.shields.io/discord/813085864355037235?color=blue&label=Discord&logo=Discord)](https://discord.gg/5SEtApPeyG) [![](https://img.shields.io/badge/Kohya%20Setting-Wiki-Blue)](https://github.com/bmaltais/kohya_ss/wiki/LoRA-training-parameters) [![](https://img.shields.io/badge/Khoá%20học-SD-red)](https://hungdiffusion.com/)

---
# 💡 Hướng dẫn sử dụng :


 - B1 : Chuẩn bị thư mục ảnh mẫu để train theo đúng kích thước, tải lên drive, có thể phân chia thư mục con
 - B1 : Chạy phần 1
 - B2 : Chỉnh Path để dẫn đến thư mục data đã tải và chạy phần 2
 - B3 : Chỉnh các thông số train tại mục 3 và chạy phần 3
 - B4 : Chạy mục 4 và theo dõi kết quả train
 ---

 🔻 : Lưu ý quan trọng thay đổi theo mỗi lần train

 🔸 : Giá trị tham khảo, có thể để mặc định


# 🔌 1. Cài đặt nền

In [ ]:
# @title Install
from google.colab import drive
drive.mount('/content/drive')

root_dir = "/content"
repo_dir = f"{root_dir}/Kohya-Colab"
training_dir = f"{root_dir}/Train_Config"
pretrained_model = f"{root_dir}/Train_model"
config_dir = f"{training_dir}/Config"

accelerate_config = f"{repo_dir}/accelerate_config/config.yaml"
tools_dir = f"{repo_dir}/tools"
finetune_dir = f"{repo_dir}/finetune"

!git clone https://github.com/StableDiffusionVN/SDVN-kohya-colab-sdxl {repo_dir}

%run {repo_dir}/TrainScript.ipynb

install_sdxl()

aria_down("https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors",pretrained_model,"sdxl_vae.safetensors")
vae_path = f"{pretrained_model}/sdxl_vae.safetensors"

# 📂 2. Kết nối data Train

In [ ]:
#@markdown ##<br>🔻 2.1 Đường dẫn thư mục dữ liệu trong drive

#@markdown <br>💡 `Có thể bỏ trống RegFolder`
TrainFolder = "/content/drive/MyDrive/SD-Data/TrainData"  # @param {type:'string'}
DataClean = False #@param {type:"boolean"}
Merge_metadata = True #@param {type:"boolean"}
SubFolder = True
#@markdown ##<br>🔸 2.2 Tạo caption tự động
AutoCaption = "All" # @param ["none", "All", "BLIP-Caption", "Waifu-1.4"]
Caption_Length = "Medium" # @param ['Short','Medium','Long']

# @markdown 🟡 `Thêm caption tuỳ chọn`
Custom_Caption = "" # @param {type:'string',placeholder:"Nhập từ khoá tuỳ chọn hoặc chuỗi muốn xoá khi chọn chế độ Remove"}
Remove_Caption = False #@param {type:"boolean"}
Append = False
# @markdown 🟡 `Tự động nhận tên thư mục con làm caption`
AddFolderName = False #@param {type:"boolean"}
SubFolder = True

extension = ".txt"

Cap_prompt = {
    'Short':['<CAPTION>',10,30,0.8],
    'Medium':['<DETAILED_CAPTION>',10,100,0.5],
    'Long':['<MORE_DETAILED_CAPTION>',10,150,0.3]
}

if DataClean == True :
    %cd {root_dir}
    clean_directory(TrainFolder)

if AutoCaption in ["Waifu-1.4","All"] :
    %cd {finetune_dir}
    config = {
        "_train_data_dir": TrainFolder,
        "batch_size": 6,
        "repo_id": "SmilingWolf/wd-v1-4-convnext-tagger-v2",
        "recursive": True,
        "remove_underscore": True,
        "general_threshold": Cap_prompt[Caption_Length][3],
        "character_threshold": 0.2,
        "caption_extension": ".txt",
        "max_data_loader_n_workers": 2,
        "debug": True,
        "undesired_tags": ""
    }
    final_args = f"python {finetune_dir}/tag_images_by_wd14_tagger.py {join_arg(config)}"
    print(final_args)
    !{final_args}

if AutoCaption in ["BLIP-Caption","All"] :
    %cd {finetune_dir}
    config = {
        "_train_data_dir" : TrainFolder,
        "batch_size" : 6,
        "beam_search" : True,
        "min_length" : Cap_prompt[Caption_Length][1],
        "max_length" : Cap_prompt[Caption_Length][2],
        "debug" : True,
        "caption_extension" : ".caption",
        "max_data_loader_n_workers" : 2,
        "recursive" : True
    }
    final_args = f"python {finetune_dir}/make_captions.py {join_arg(config)}"
    print(final_args)
    !{final_args}

if AddFolderName:
  add_forder_name(TrainFolder)
if Custom_Caption != "":
  process_dir(TrainFolder, Custom_Caption, Append, Remove_Caption)

#bucketing

bucketing_json    = f"{training_dir}/meta_lat.json"
metadata_json     = f"{training_dir}/meta_clean.json"

metadata_config = {
    "_train_data_dir": TrainFolder,
    "_out_json": metadata_json,
    "recursive": True,
    "full_path": True,
    "clean_caption": False
}

bucketing_config = {
    "_train_data_dir": TrainFolder,
    "_in_json": metadata_json,
    "_out_json": bucketing_json,
    "_model_name_or_path": vae_path,
    "recursive": True,
    "full_path": True,
    "flip_aug": False,
    "skip_existing": False,
    "batch_size": 4,
    "max_data_loader_n_workers": 2,
    "max_resolution": f"{1024}, {1024}",
    "mixed_precision": "no",
}

if Merge_metadata == True :
  %cd {finetune_dir}
  merge_metadata_args = join_arg(metadata_config)
  prepare_buckets_args = join_arg(bucketing_config)
  merge_metadata_command = f"python merge_all_to_metadata.py {merge_metadata_args}"
  prepare_buckets_command = f"python prepare_buckets_latents.py {prepare_buckets_args}"
  !{merge_metadata_command}
  !{prepare_buckets_command}

# ⚙️ 3. Cài đặt Train

In [ ]:
#@markdown ##🔻 3.1 Cài đặt chung
#@markdown <br>
Train_Option = "Lora" # @param ["Checkpoint","Lora"]
Project_name = "" #@param {type:"string",placeholder:"Nhập tên output"}
Output_Path = "/content/drive/MyDrive/SD-Data" # @param {type:"string"}
#@markdown <br>💡 `Dán link tải hoặc link đường dẫn drive để sử dụng model bất kỳ để train`
Model_Train = ""  #@param ["","SDXL-Base", "XXMix_9realisticSDXL", "LEOSAM-HelloWorldXL", "AdamXL-v3", "JuggernautXL-v9", "ZavyChromaXL-v5", "Samaritan3dCartoon-v4", "DucHaiten-AIart-SDXL", "JuggernautXL"] {allow-input: true}
Resume_Path = "" # @param {type:'string',placeholder:"Nhập tên đường dẫn thư mục State nếu muốn train tiếp từ backup"}
Save_State = False #@param {type:"boolean"}
Resolution = "1024,1024" # @param {type:'string'}
Batch_size = 1  # @param {type:"number"}
Repeats = 10  # @param {type:"number"}
Lr_scheduler = "constant_with_warmup" # @param ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup", "adafactor"] {allow-input: false}
#@markdown ##<br>🔻 3.2-A | Checkpoint Setting
#@markdown <br>
Ckpt_Lr  = 1e-6  # @param {'type':'number'}
Max_train_steps = 10000  # @param {type:"number"}
Save_Every_N_Steps = 1000  # @param {type:"number"}
Train_text_encoder = False  # @param {type:"boolean"}

#@markdown ##<br>🔻 3.2-B | Lora Setting
LoRA_Network_Weights = "" # @param {'type':'string'}
Network_dim = 32  # @param {'type':'number'}
Network_alpha = 16 # @param {'type':'number'}
Lora_Lr = 1e-4  # @param {'type':'number'}
Epochs = 4  # @param {type:"number"}
Save_Every_N_Epochs = 1  # @param {type:"number"}
Comment = "creator by sdvn.me" # @param {'type':'string'}

output_name = Project_name if Project_name != "" else "project_name"
output_dir = f'{Output_Path}/{"Lora" if Train_Option == "Lora" else "Model"}'
file_path = f'{repo_dir}/model_lib.json'
with open(file_path, 'r') as json_file:
    modellist = json.load(json_file)
pretrained_model_name_or_path = download_lib(Model_Train,modellist,pretrained_model)

resume = Resume_Path
save_state = Save_State
resolution = Resolution
train_batch_size = Batch_size
dataset_repeats = Repeats
lr_scheduler = Lr_scheduler
learning_rate = Ckpt_Lr if Train_Option == "Checkpoint" else Lora_Lr
max_train_steps = Max_train_steps if Train_Option == "Checkpoint" else None
save_every_n_steps = Save_Every_N_Steps if Train_Option == "Checkpoint" else None
train_text_encoder = Train_text_encoder
network_weights = LoRA_Network_Weights
network_dim = Network_dim
network_alpha = Network_alpha
max_train_epochs = Epochs
save_every_n_epochs = Save_Every_N_Epochs
training_comment = Comment
with open(f'{training_dir}/meta_clean.json', 'r') as json_file:
    data = json.load(json_file)
prompt = list(data.values())[0]["tags"]
sampler_prompt = f"{prompt}, best quality"
subset = [{"prompt": sampler_prompt}]
%cd {root_dir}

sample_file   = f"{config_dir}/sample_prompt.toml"
config_file     = f"{config_dir}/config_file.toml"

default_config_lora = f"{repo_dir}/config/default_config_lora.json"
default_config_checkpoint = f"{repo_dir}/config/default_config_checkpoint.json"
default_config_sampler = f"{repo_dir}/config/default_config_sampler.json"

final_config(default_config_lora if Train_Option == "Lora" else default_config_checkpoint,config_file)
final_config(default_config_sampler,sample_file)

print("Done")

# ⌛️ 4. Run

In [ ]:
# @title
#@markdown #▶️ Theo dõi quá trình và kiểm soát chất lượng
#@markdown *Kiểm tra sample tại ```{Output_Path}/sample```*
#@markdown <br>*Có thể dừng bất cứ khi nào sample thấy ưng ý*
AutoDisconect = False #@param {type:"boolean"}

%cd {repo_dir}

accelerate_conf = {
    "config_file" : accelerate_config,
    "num_cpu_threads_per_process" : 1,
}

train_conf = {
    "sample_prompts"  : sample_file,
    "config_file"     : config_file
}

accelerate_args = join_arg(accelerate_conf)
train_args = join_arg(train_conf)
Train_File = "sdxl_train_network.py" if Train_Option == "Lora" else "sdxl_train.py"

final_args = f"accelerate launch {accelerate_args} {Train_File} {train_args}"
!{final_args}
if AutoDisconect:
    from time import sleep
    sleep(180)
    from google.colab import runtime
    runtime.unassign()